Here's an extensive summary in markdown format, drawing on the provided sources, our conversation history, and with some bolding to highlight key concepts:

**Structured Output from Language Models in LangChain**

*   **Purpose:** This guide focuses on methods for getting structured outputs from language models (LLMs), which is often needed for tasks such as extracting data to insert into a database or use in a downstream system.
*   **Key Methods:** There are multiple ways to achieve structured output, including using the `.with_structured_output()` method, and by directly prompting and parsing model outputs.
*  **Prerequisites:** Familiarity with chat models and function/tool calling is recommended before using these techniques.

**The `.with_structured_output()` Method**

*   **Supported Models:** This method is available for models that have native APIs for structuring outputs, like tool/function calling or JSON mode.
*   **Schema Input:** This method requires a schema that specifies the names, types, and descriptions of the desired output attributes. The schema can be a `TypedDict` class, a JSON Schema, or a Pydantic class.
    *   **TypedDict/JSON Schema**: When using `TypedDict` or JSON Schema, the method returns a dictionary.
    *   **Pydantic Class:** When using a Pydantic class, the method returns a Pydantic object.
*   **Validation:** A key advantage of using Pydantic is that the model-generated output will be validated. Pydantic will raise an error if required fields are missing or of the wrong type.

**Using Pydantic Classes**

*   **Defining the Schema:** Define the desired structure using a Pydantic class, including field names, types, and descriptions. The class name, docstring, and parameter descriptions are important, as this information is added to the model prompt.
*   **Example:**
    ```python
    from typing import Optional
    from pydantic import BaseModel, Field

    class Joke(BaseModel):
        """Joke to tell user."""
        setup: str = Field(description="The setup of the joke")
        punchline: str = Field(description="The punchline to the joke")
        rating: Optional[int] = Field(default=None, description="How funny the joke is, from 1 to 10")
    ```
*   **Usage:**
    ```python
    structured_llm = llm.with_structured_output(Joke)
    structured_llm.invoke("Tell me a joke about cats")
    ```

**Using TypedDict or JSON Schema**

*   **TypedDict:** Use `TypedDict` if you don't want Pydantic's validation or want to stream outputs. You can optionally use the `Annotated` syntax to specify default values and descriptions.
*   **JSON Schema:** A JSON Schema dictionary can also define the structure. This is more verbose but makes it very clear how each parameter is documented.
*   **Example (TypedDict):**
    ```python
    from typing_extensions import Annotated, TypedDict
    from typing import Optional

    class Joke(TypedDict):
        """Joke to tell user."""
        setup: Annotated[str, ..., "The setup of the joke"]
        punchline: Annotated[str, ..., "The punchline of the joke"]
        rating: Annotated[Optional[int], None, "How funny the joke is, from 1 to 10"]
    ```
*   **Example (JSON Schema):**
    ```python
    json_schema = {
        "title": "joke",
        "description": "Joke to tell user.",
        "type": "object",
        "properties": {
            "setup": {
                "type": "string",
                "description": "The setup of the joke",
            },
            "punchline": {
                "type": "string",
                "description": "The punchline to the joke",
            },
            "rating": {
                "type": "integer",
                "description": "How funny the joke is, from 1 to 10",
                "default": None,
            },
        },
        "required": ["setup", "punchline"],
    }
    ```

**Choosing Between Multiple Schemas**

*   **Union-Typed Attribute:** You can let the model choose between multiple schemas by creating a parent schema with a `Union`-typed attribute.
*   **Example:**
    ```python
    from typing import Union
    from pydantic import BaseModel, Field

    class Joke(BaseModel):
        """Joke to tell user."""
        setup: str = Field(description="The setup of the joke")
        punchline: str = Field(description="The punchline to the joke")
        rating: Optional[int] = Field(default=None, description="How funny the joke is, from 1 to 10")

    class ConversationalResponse(BaseModel):
        """Respond in a conversational manner. Be kind and helpful."""
        response: str = Field(description="A conversational response to the user's query")

    class FinalResponse(BaseModel):
        final_output: Union[Joke, ConversationalResponse]
    ```

**Streaming**

*   **Supported Types:** Streaming is supported when the output type is a dictionary (i.e., when the schema is specified as a `TypedDict` class or JSON Schema dictionary).
*   **Output:** Streaming yields aggregated chunks, not deltas.

**Few-Shot Prompting**

*   **Examples in System Message:** You can add examples to a system message to guide the model, especially for complex schemas.
*   **Examples as Tool Calls:**  If the model uses tool calling, you can pass examples as explicit tool calls. This can sometimes lead to better performance.

**Advanced Options**

*   **Specifying Method:** For models supporting multiple output structuring methods, you can specify which method to use (e.g., `method="json_mode"`).
*   **Raw Outputs:** By using `include_raw=True`, you can access the raw model output, parsed value, and any errors, allowing you to handle parsing failures.

**Prompting and Parsing Model Outputs Directly**

*   **When to Use:** For models that do not support `.with_structured_output()`, or when you need more control, you can directly prompt the model to use a specific format and use an output parser.
*   **PydanticOutputParser:** This built-in parser can parse model outputs that match a Pydantic schema. It can add formatting instructions directly to the prompt.
    *   **Example:**
        ```python
        from langchain_core.output_parsers import PydanticOutputParser
        from langchain_core.prompts import ChatPromptTemplate
        from pydantic import BaseModel, Field
        class Person(BaseModel):
            """Information about a person."""
            name: str = Field(..., description="The name of the person")
            height_in_meters: float = Field(..., description="The height of the person expressed in meters.")
        class People(BaseModel):
            """Identifying information about all people in a text."""
            people: List[Person]
        parser = PydanticOutputParser(pydantic_object=People)
        prompt = ChatPromptTemplate.from_messages([
            ("system", "Answer the user query. Wrap the output in `json` tags\n{format_instructions}"),
            ("human", "{query}"),
        ]).partial(format_instructions=parser.get_format_instructions())
        ```
*  **Custom Parsing:** You can also create custom prompts and parsers using LangChain Expression Language (LCEL) and plain functions to parse the output from the model.
    *  **Example:**
        ```python
        import json
        import re
        from langchain_core.messages import AIMessage
        def extract_json(message: AIMessage) -> List[dict]:
            text = message.content
            pattern = r"\`\`\`json(.*?)\`\`\`"
            matches = re.findall(pattern, text, re.DOTALL)
            try:
                return [json.loads(match.strip()) for match in matches]
            except Exception:
                raise ValueError(f"Failed to parse: {message}")
        ```

**Key Takeaways**

*   The `.with_structured_output()` method is the easiest way to get structured outputs, for models that support it.
*   Pydantic classes provide data validation, while TypedDict and JSON Schema offer flexibility and streaming options.
*   Few-shot prompting is crucial for complex schemas, especially when using tool calling.
*   For models without native structured output support, you can use output parsers or create a custom solution.
*   For complex situations with models that may not perfectly adhere to format instructions, using raw outputs (`include_raw=True`) can enable more robust output handling and error processing.

This summary provides a comprehensive overview of how to return structured data from language models using LangChain, covering different approaches, their advantages, and use cases, drawing on the information in the provided sources.


In [1]:
import getpass
import os

if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [2]:
from typing import Optional

from pydantic import BaseModel, Field


# Pydantic
class Joke(BaseModel):
    """Joke to tell user."""

    setup: str = Field(description="The setup of the joke")
    punchline: str = Field(description="The punchline to the joke")
    rating: Optional[int] = Field(
        default=None, description="How funny the joke is, from 1 to 10"
    )


structured_llm = llm.with_structured_output(Joke)

structured_llm.invoke("Tell me a joke about cats")

Joke(setup='Why was the cat sitting on the computer?', punchline='Because it wanted to keep an eye on the mouse!', rating=8)

In [5]:
from typing_extensions import Annotated, TypedDict


# TypedDict
class Joke(TypedDict):
    """Joke to tell user."""

    setup: Annotated[str, ..., "The setup of the joke"]

    # Alternatively, we could have specified setup as:

    # setup: str                    # no default, no description
    # setup: Annotated[str, ...]    # no default, no description
    # setup: Annotated[str, "foo"]  # default, no description

    punchline: Annotated[str, ..., "The punchline of the joke"]
    rating: Annotated[Optional[int], None, "How funny the joke is, from 1 to 10"]


structured_llm = llm.with_structured_output(Joke)

structured_llm.invoke("Tell me a joke about a dog")

{'setup': 'Why did the dog sit in the shade?',
 'punchline': "Because he didn't want to become a hot dog!",
 'rating': 7}

In [8]:
json_schema = {
    "title": "joke",
    "description": "Joke to tell user.",
    "type": "object",
    "properties": {
        "setup": {
            "type": "string",
            "description": "The setup of the joke",
        },
        "punchline": {
            "type": "string",
            "description": "The punchline to the joke",
        },
        "rating": {
            "type": "integer",
            "description": "How funny the joke is, from 1 to 10",
            "default": None,
        },
    },
    "required": ["setup", "punchline"],
}
structured_llm = llm.with_structured_output(json_schema)

structured_llm.invoke("Tell me a joke about a space")

{'setup': 'Why did the sun go to school?',
 'punchline': 'To get a little brighter!',
 'rating': 7}

In [11]:
# The simplest way to let the model choose from multiple schemas is to create a parent schema that has a Union-typed attribute.

from typing import Union


class Joke(BaseModel):
    """Joke to tell user."""

    setup: str = Field(description="The setup of the joke")
    punchline: str = Field(description="The punchline to the joke")
    rating: Optional[int] = Field(
        default=None, description="How funny the joke is, from 1 to 10"
    )


class ConversationalResponse(BaseModel):
    """Respond in a conversational manner. Be kind and helpful."""

    response: str = Field(description="A conversational response to the user's query")


class FinalResponse(BaseModel):
    final_output: Union[Joke, ConversationalResponse]


structured_llm = llm.with_structured_output(FinalResponse)

structured_llm.invoke("Tell me a joke about job serach process")


FinalResponse(final_output=Joke(setup='Why did the job seeker bring a ladder to the interview?', punchline='Because they heard the job was at a higher level!', rating=7))

In [12]:
structured_llm.invoke("How are you today?")

FinalResponse(final_output=ConversationalResponse(response="I'm just a bunch of code, but I'm here and ready to help you! How about you? How's your day going?"))

In [18]:
from langchain_ollama import ChatOllama

ollama = ChatOllama(model='llama3.1')

structured_ollama = ollama.with_structured_output(Joke)
structured_ollama.invoke("Tell me a joke about job serach process")



Joke(setup='Why did the job seeker bring a ladder to the interview?', punchline='Because they wanted to reach their career goals!', rating=8)

In [19]:
from typing_extensions import Annotated, TypedDict


# TypedDict
class Joke(TypedDict):
    """Joke to tell user."""

    setup: Annotated[str, ..., "The setup of the joke"]
    punchline: Annotated[str, ..., "The punchline of the joke"]
    rating: Annotated[Optional[int], None, "How funny the joke is, from 1 to 10"]


structured_ollama = ollama.with_structured_output(Joke)

for chunk in structured_ollama.stream("Tell me a joke about cats"):
    print(chunk)

{'punchline': 'Why did the cat join a band? Because it wanted to be the purr-cussionist!', 'rating': 8, 'setup': 'A cat decided to audition for a band and had to choose an instrument.'}


In [22]:
for chunk in ollama.stream("Tell me a joke about cats"):
    print(chunk.content)

Why
 did
 the
 cat
 join
 a
 band
?


Because
 it
 wanted
 to
 be
 the
 pur
r
-c
ussion
ist
.



In [20]:
from typing_extensions import Annotated, TypedDict


# TypedDict
class Joke(TypedDict):
    """Joke to tell user."""

    setup: Annotated[str, ..., "The setup of the joke"]
    punchline: Annotated[str, ..., "The punchline of the joke"]
    rating: Annotated[Optional[int], None, "How funny the joke is, from 1 to 10"]


structured_ollama = llm.with_structured_output(Joke)

for chunk in structured_ollama.stream("Tell me a joke about cats"):
    print(chunk)

{}
{'setup': ''}
{'setup': 'Why'}
{'setup': 'Why was'}
{'setup': 'Why was the'}
{'setup': 'Why was the cat'}
{'setup': 'Why was the cat sitting'}
{'setup': 'Why was the cat sitting on'}
{'setup': 'Why was the cat sitting on the'}
{'setup': 'Why was the cat sitting on the computer'}
{'setup': 'Why was the cat sitting on the computer?'}
{'setup': 'Why was the cat sitting on the computer?', 'punchline': ''}
{'setup': 'Why was the cat sitting on the computer?', 'punchline': 'Because'}
{'setup': 'Why was the cat sitting on the computer?', 'punchline': 'Because it'}
{'setup': 'Why was the cat sitting on the computer?', 'punchline': 'Because it wanted'}
{'setup': 'Why was the cat sitting on the computer?', 'punchline': 'Because it wanted to'}
{'setup': 'Why was the cat sitting on the computer?', 'punchline': 'Because it wanted to keep'}
{'setup': 'Why was the cat sitting on the computer?', 'punchline': 'Because it wanted to keep an'}
{'setup': 'Why was the cat sitting on the computer?', 'punc

In [23]:
from langchain_core.prompts import ChatPromptTemplate

system = """You are a hilarious comedian. Your specialty is knock-knock jokes. \
Return a joke which has the setup (the response to "Who's there?") and the final punchline (the response to "<setup> who?").

Here are some examples of jokes:

example_user: Tell me a joke about planes
example_assistant: {{"setup": "Why don't planes ever get tired?", "punchline": "Because they have rest wings!", "rating": 2}}

example_user: Tell me another joke about planes
example_assistant: {{"setup": "Cargo", "punchline": "Cargo 'vroom vroom', but planes go 'zoom zoom'!", "rating": 10}}

example_user: Now about caterpillars
example_assistant: {{"setup": "Caterpillar", "punchline": "Caterpillar really slow, but watch me turn into a butterfly and steal the show!", "rating": 5}}"""

prompt = ChatPromptTemplate.from_messages([("system", system), ("human", "{input}")])

few_shot_structured_llm = prompt | structured_llm
few_shot_structured_llm.invoke("what's something funny about woodpeckers")

FinalResponse(final_output=Joke(setup='Woodpecker', punchline="Woodpecker's favorite music? Anything with a good 'beat'!", rating=7))

In [24]:
from langchain_core.prompts import ChatPromptTemplate

system = """You are a hilarious comedian. Your specialty is knock-knock jokes. \
Return a joke which has the setup (the response to "Who's there?") and the final punchline (the response to "<setup> who?").

Here are some examples of jokes:

example_user: Tell me a joke about planes
example_assistant: {{"setup": "Why don't planes ever get tired?", "punchline": "Because they have rest wings!", "rating": 2}}

example_user: Tell me another joke about planes
example_assistant: {{"setup": "Cargo", "punchline": "Cargo 'vroom vroom', but planes go 'zoom zoom'!", "rating": 10}}

example_user: Now about caterpillars
example_assistant: {{"setup": "Caterpillar", "punchline": "Caterpillar really slow, but watch me turn into a butterfly and steal the show!", "rating": 5}}"""

prompt = ChatPromptTemplate.from_messages([("system", system), ("human", "{input}")])

few_shot_structured_llm = prompt | structured_ollama
few_shot_structured_llm.invoke("what's something funny about woodpeckers")

{'setup': 'Woodpecker',
 'punchline': "Woodpecker knock on wood, but I just knock knock because I'm too shy to say hello!",
 'rating': 7}

In [25]:
from langchain_core.messages import AIMessage, HumanMessage, ToolMessage

examples = [
    HumanMessage("Tell me a joke about planes", name="example_user"),
    AIMessage(
        "",
        name="example_assistant",
        tool_calls=[
            {
                "name": "joke",
                "args": {
                    "setup": "Why don't planes ever get tired?",
                    "punchline": "Because they have rest wings!",
                    "rating": 2,
                },
                "id": "1",
            }
        ],
    ),
    # Most tool-calling models expect a ToolMessage(s) to follow an AIMessage with tool calls.
    ToolMessage("", tool_call_id="1"),
    # Some models also expect an AIMessage to follow any ToolMessages,
    # so you may need to add an AIMessage here.
    HumanMessage("Tell me another joke about planes", name="example_user"),
    AIMessage(
        "",
        name="example_assistant",
        tool_calls=[
            {
                "name": "joke",
                "args": {
                    "setup": "Cargo",
                    "punchline": "Cargo 'vroom vroom', but planes go 'zoom zoom'!",
                    "rating": 10,
                },
                "id": "2",
            }
        ],
    ),
    ToolMessage("", tool_call_id="2"),
    HumanMessage("Now about caterpillars", name="example_user"),
    AIMessage(
        "",
        tool_calls=[
            {
                "name": "joke",
                "args": {
                    "setup": "Caterpillar",
                    "punchline": "Caterpillar really slow, but watch me turn into a butterfly and steal the show!",
                    "rating": 5,
                },
                "id": "3",
            }
        ],
    ),
    ToolMessage("", tool_call_id="3"),
]
system = """You are a hilarious comedian. Your specialty is knock-knock jokes. \
Return a joke which has the setup (the response to "Who's there?") \
and the final punchline (the response to "<setup> who?")."""

prompt = ChatPromptTemplate.from_messages(
    [("system", system), ("placeholder", "{examples}"), ("human", "{input}")]
)
few_shot_structured_llm = prompt | structured_llm
few_shot_structured_llm.invoke({"input": "crocodiles", "examples": examples})

FinalResponse(final_output=Joke(setup='Crocodile', punchline="Crocodile, you know, the only one who could never win at poker because he's always in a snap!", rating=7))

In [26]:
few_shot_structured_llm = prompt | structured_ollama
few_shot_structured_llm.invoke({"input": "crocodiles", "examples": examples})

{'setup': 'Crocodile',
 'punchline': 'Crocodile tears? More like crocodile cheers, when they finally catch their lunch!',
 'rating': 6}

In [27]:
structured_llm = llm.with_structured_output(None, method="json_mode")

structured_llm.invoke(
    "Tell me a joke about cats, respond in JSON with `setup` and `punchline` keys"
)


{'setup': 'Why was the cat sitting on the computer?',
 'punchline': 'Because it wanted to keep an eye on the mouse!'}

In [28]:
structured_llm = ollama.with_structured_output(None, method="json_mode")

structured_llm.invoke(
    "Tell me a joke about cats, respond in JSON with `setup` and `punchline` keys"
)


ValueError: Received unsupported arguments {'method': 'json_mode'}

In [29]:
structured_llm = llm.with_structured_output(Joke, include_raw=True)

structured_llm.invoke("Tell me a joke about cats")

{'raw': AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_lzgyWp9o3kAbBl9qrx3ffinW', 'function': {'arguments': '{"setup":"Why was the cat sitting on the computer?","punchline":"Because it wanted to keep an eye on the mouse!","rating":7}', 'name': 'Joke'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 34, 'prompt_tokens': 93, 'total_tokens': 127, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0aa8d3e20b', 'finish_reason': 'stop', 'logprobs': None}, id='run-6d619156-bccf-4511-aaa3-c9a9d0c90ea5-0', tool_calls=[{'name': 'Joke', 'args': {'setup': 'Why was the cat sitting on the computer?', 'punchline': 'Because it wanted to keep an eye on the mouse!', 'rating': 7}, 'id': 'call_lzgyWp9o3kAbBl9qrx3ffinW'

In [30]:
structured_llm = ollama.with_structured_output(Joke, include_raw=True)

structured_llm.invoke("Tell me a joke about cats")

{'raw': AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.1', 'created_at': '2024-12-22T17:45:24.458772318Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1814635029, 'load_duration': 1320654108, 'prompt_eval_count': 206, 'prompt_eval_duration': 77000000, 'eval_count': 55, 'eval_duration': 415000000, 'message': Message(role='assistant', content='', images=None, tool_calls=[ToolCall(function=Function(name='Joke', arguments={'punchline': 'Why did the cat join a band? Because it wanted to be a purr-cussionist!', 'rating': 7, 'setup': 'A cat walks into a music store'}))])}, id='run-d5df0771-042b-4dd4-9337-32fe323f70a5-0', tool_calls=[{'name': 'Joke', 'args': {'punchline': 'Why did the cat join a band? Because it wanted to be a purr-cussionist!', 'rating': 7, 'setup': 'A cat walks into a music store'}, 'id': 'fd02bce7-8274-49de-b4ea-c64df1a4b001', 'type': 'tool_call'}], usage_metadata={'input_tokens': 206, 'output_tokens': 55, 'total_tokens': 261}),
 

In [33]:
structured_llm.invoke("Tell me a joke about cats")["raw"]

AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.1', 'created_at': '2024-12-22T17:47:00.04372062Z', 'done': True, 'done_reason': 'stop', 'total_duration': 423438947, 'load_duration': 16147204, 'prompt_eval_count': 206, 'prompt_eval_duration': 2000000, 'eval_count': 54, 'eval_duration': 404000000, 'message': Message(role='assistant', content='', images=None, tool_calls=[ToolCall(function=Function(name='Joke', arguments={'punchline': 'Why did the cat join a band? Because it wanted to be the purr-cussionist!', 'rating': 7, 'setup': 'A cat joins a band.'}))])}, id='run-7df9ef25-2543-441a-a803-957f88c1f1de-0', tool_calls=[{'name': 'Joke', 'args': {'punchline': 'Why did the cat join a band? Because it wanted to be the purr-cussionist!', 'rating': 7, 'setup': 'A cat joins a band.'}, 'id': '049a24f2-0ceb-46f8-a665-8013831c8791', 'type': 'tool_call'}], usage_metadata={'input_tokens': 206, 'output_tokens': 54, 'total_tokens': 260})

In [34]:
from typing import List

from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field


class Person(BaseModel):
    """Information about a person."""

    name: str = Field(..., description="The name of the person")
    height_in_meters: float = Field(
        ..., description="The height of the person expressed in meters."
    )


class People(BaseModel):
    """Identifying information about all people in a text."""

    people: List[Person]


# Set up a parser
parser = PydanticOutputParser(pydantic_object=People)

# Prompt
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Answer the user query. Wrap the output in `json` tags\n{format_instructions}",
        ),
        ("human", "{query}"),
    ]
).partial(format_instructions=parser.get_format_instructions())

In [36]:
print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"$defs": {"Person": {"description": "Information about a person.", "properties": {"name": {"description": "The name of the person", "title": "Name", "type": "string"}, "height_in_meters": {"description": "The height of the person expressed in meters.", "title": "Height In Meters", "type": "number"}}, "required": ["name", "height_in_meters"], "title": "Person", "type": "object"}}, "description": "Identifying information about all people in a text.", "properties": {"people": {"items": {"$ref": "#/$defs/Person"}, "title": "People", "type": "arra

In [37]:
query = "Anna is 23 years old and she is 6 feet tall"

print(prompt.invoke({"query": query}).to_string())

System: Answer the user query. Wrap the output in `json` tags
The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"$defs": {"Person": {"description": "Information about a person.", "properties": {"name": {"description": "The name of the person", "title": "Name", "type": "string"}, "height_in_meters": {"description": "The height of the person expressed in meters.", "title": "Height In Meters", "type": "number"}}, "required": ["name", "height_in_meters"], "title": "Person", "type": "object"}}, "description": "Identifying information about all people in a text.", "properties": {"people": {"items"

In [38]:
chain = prompt | llm | parser

chain.invoke({"query": query})

People(people=[Person(name='Anna', height_in_meters=1.8288)])

In [42]:
chain = prompt | ollama | parser

chain.invoke({"query": query})

People(people=[Person(name='Anna', height_in_meters=1.8288)])

In [43]:
import json
import re
from typing import List

from langchain_core.messages import AIMessage
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field


class Person(BaseModel):
    """Information about a person."""

    name: str = Field(..., description="The name of the person")
    height_in_meters: float = Field(
        ..., description="The height of the person expressed in meters."
    )


class People(BaseModel):
    """Identifying information about all people in a text."""

    people: List[Person]


# Prompt
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Answer the user query. Output your answer as JSON that  "
            "matches the given schema: \`\`\`json\n{schema}\n\`\`\`. "
            "Make sure to wrap the answer in \`\`\`json and \`\`\` tags",
        ),
        ("human", "{query}"),
    ]
).partial(schema=People.schema())


# Custom parser
def extract_json(message: AIMessage) -> List[dict]:
    """Extracts JSON content from a string where JSON is embedded between \`\`\`json and \`\`\` tags.

    Parameters:
        text (str): The text containing the JSON content.

    Returns:
        list: A list of extracted JSON strings.
    """
    text = message.content
    # Define the regular expression pattern to match JSON blocks
    pattern = r"\`\`\`json(.*?)\`\`\`"

    # Find all non-overlapping matches of the pattern in the string
    matches = re.findall(pattern, text, re.DOTALL)

    # Return the list of matched JSON strings, stripping any leading or trailing whitespace
    try:
        return [json.loads(match.strip()) for match in matches]
    except Exception:
        raise ValueError(f"Failed to parse: {message}")

/tmp/ipykernel_687156/1836477667.py:36: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  ).partial(schema=People.schema())


In [44]:
query = "Anna is 23 years old and she is 6 feet tall"

print(prompt.format_prompt(query=query).to_string())

System: Answer the user query. Output your answer as JSON that  matches the given schema: \`\`\`json
{'$defs': {'Person': {'description': 'Information about a person.', 'properties': {'name': {'description': 'The name of the person', 'title': 'Name', 'type': 'string'}, 'height_in_meters': {'description': 'The height of the person expressed in meters.', 'title': 'Height In Meters', 'type': 'number'}}, 'required': ['name', 'height_in_meters'], 'title': 'Person', 'type': 'object'}}, 'description': 'Identifying information about all people in a text.', 'properties': {'people': {'items': {'$ref': '#/$defs/Person'}, 'title': 'People', 'type': 'array'}}, 'required': ['people'], 'title': 'People', 'type': 'object'}
\`\`\`. Make sure to wrap the answer in \`\`\`json and \`\`\` tags
Human: Anna is 23 years old and she is 6 feet tall


In [45]:
chain = prompt | llm | extract_json

chain.invoke({"query": query})



[{'people': [{'name': 'Anna', 'height_in_meters': 1.8288}]}]

In [46]:
chain = prompt | ollama | extract_json

chain.invoke({"query": query})



[{'$defs': {'Person': {'description': 'Information about a person.',
    'properties': {'name': {'description': 'The name of the person',
      'title': 'Name',
      'type': 'string'},
     'height_in_meters': {'description': 'The height of the person expressed in meters.',
      'title': 'Height In Meters',
      'type': 'number'}},
    'required': ['name', 'height_in_meters'],
    'title': 'Person',
    'type': 'object'}},
  'description': 'Identifying information about all people in a text.',
  'properties': {'people': {'items': {'$ref': '#/$defs/Person'},
    'title': 'People',
    'type': 'array'}},
  'required': ['people'],
  'title': 'People',
  'type': 'object'}]